# Chat GPT

In [ ]:
#| default_exp bigram

In [ ]:
#|export
import requests
from fastcore.all import *
from rich import print
import numpy
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

<torch._C.Generator>

In [ ]:
#|export
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Read & Explore Data

In [ ]:
#|export
path = urlsave('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt', '../data/input.txt')

In [ ]:
#|export
text = path.read_text(encoding='utf-8')

In [ ]:
print("Length of the dataset ", len(text))

Length of the dataset  1115394

In [ ]:
# lets look at the first 1000 characters 
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.

## Tokenization

In [ ]:
#|export
chars = L(set(text)).sorted()
vocab_size = len(chars)

In [ ]:
print(''.join(chars), vocab_size)

!$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65

In [ ]:
#|export
stoi = {ch: i for i, ch in enumerate(chars)}
itos = dict(map(reversed, stoi.items()))
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]

hii there

In [ ]:
#|export
# lets now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earlier will look like this to the GPT

torch.Size([1115394])
torch.int64

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 46, 47, 43, 44,  1, 43, 52, 43, 51, 63,
         1, 58, 53,  1, 58, 46, 43,  1, 54, 43, 53, 54, 50, 43,  8,  0,  0, 13,
        50, 50, 10,  0, 35, 43,  1, 49, 52, 53, 61,  5, 58,  6,  1, 61, 43,  1,
        49, 52, 53, 61,  5, 58,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47, 58,
        47, 64, 43, 52, 10,  0, 24, 43, 58,  1, 59, 57,  1, 49, 47, 50, 50,  1,
        46, 47, 51,  6,  1, 39, 52, 42,  1, 61, 43,  5, 50, 50,  1, 46, 39, 60,
        43,  1, 41, 53, 56, 52,  1, 39, 58,  1, 53, 59, 56,  1, 53, 61, 52,  1,
        54, 56, 47, 41, 43,  8,  0, 21, 57,  5, 58,  1, 39,  1, 60, 43, 56, 42,
        47, 41, 58, 12,  0,  0, 13, 50, 50, 10,  0, 26, 53,  1, 51, 53, 56, 43,
         1, 58, 39, 50, 49, 47, 52, 45,  1, 53, 52,  5, 58, 11,  1, 50, 43, 58,
         1, 47, 58,  1, 40, 43,  1, 42, 53, 52, 43, 10,  1, 39, 61, 39, 63,  6,
         1, 39, 61, 39, 63,  2,  0,  0, 31, 43, 41, 53, 52, 42,  1, 15, 47, 58,
        47, 64, 43, 52, 10,  0, 27, 52, 43,  1, 61, 53, 56, 42,  6,  1, 45, 53,
        53, 42,  1, 41, 47, 58, 47, 64, 43, 52, 57,  8,  0,  0, 18, 47, 56, 57,
        58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 35, 43,  1, 39, 56, 43,  1,
        39, 41, 41, 53, 59, 52, 58, 43, 42,  1, 54, 53, 53, 56,  1, 41, 47, 58,
        47, 64, 43, 52, 57,  6,  1, 58, 46, 43,  1, 54, 39, 58, 56, 47, 41, 47,
        39, 52, 57,  1, 45, 53, 53, 42,  8,  0, 35, 46, 39, 58,  1, 39, 59, 58,
        46, 53, 56, 47, 58, 63,  1, 57, 59, 56, 44, 43, 47, 58, 57,  1, 53, 52,
         1, 61, 53, 59, 50, 42,  1, 56, 43, 50, 47, 43, 60, 43,  1, 59, 57, 10,
         1, 47, 44,  1, 58, 46, 43, 63,  0, 61, 53, 59, 50, 42,  1, 63, 47, 43,
        50, 42,  1, 59, 57,  1, 40, 59, 58,  1, 58, 46, 43,  1, 57, 59, 54, 43,
        56, 44, 50, 59, 47, 58, 63,  6,  1, 61, 46, 47, 50, 43,  1, 47, 58,  1,
        61, 43, 56, 43,  0, 61, 46, 53, 50, 43, 57, 53, 51, 43,  6,  1, 61, 43,
         1, 51, 47, 45, 46, 58,  1, 45, 59, 43, 57, 57,  1, 58, 46, 43, 63,  1,
        56, 43, 50, 47, 43, 60, 43, 42,  1, 59, 57,  1, 46, 59, 51, 39, 52, 43,
        50, 63, 11,  0, 40, 59, 58,  1, 58, 46, 43, 63,  1, 58, 46, 47, 52, 49,
         1, 61, 43,  1, 39, 56, 43,  1, 58, 53, 53,  1, 42, 43, 39, 56, 10,  1,
        58, 46, 43,  1, 50, 43, 39, 52, 52, 43, 57, 57,  1, 58, 46, 39, 58,  0,
        39, 44, 44, 50, 47, 41, 58, 57,  1, 59, 57,  6,  1, 58, 46, 43,  1, 53,
        40, 48, 43, 41, 58,  1, 53, 44,  1, 53, 59, 56,  1, 51, 47, 57, 43, 56,
        63,  6,  1, 47, 57,  1, 39, 57,  1, 39, 52,  0, 47, 52, 60, 43, 52, 58,
        53, 56, 63,  1, 58, 53,  1, 54, 39, 56, 58, 47, 41, 59, 50, 39, 56, 47,
        57, 43,  1, 58, 46, 43, 47, 56,  1, 39, 40, 59, 52, 42, 39, 52, 41, 43,
        11,  1, 53, 59, 56,  0, 57, 59, 44, 44, 43, 56, 39, 52, 41, 43,  1, 47,
        57,  1, 39,  1, 45, 39, 47, 52,  1, 58, 53,  1, 58, 46, 43, 51,  1, 24,
        43, 58,  1, 59, 57,  1, 56, 43, 60, 43, 52, 45, 43,  1, 58, 46, 47, 57,
         1, 61, 47, 58, 46,  0, 53, 59, 56,  1, 54, 47, 49, 43, 57,  6,  1, 43,


## Train/Validation Split

In [ ]:
#|export
# lets now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest will be val
train_data = data[:n]
val_data = data[n:]

## Dataloader: Batches of chunks of data

In [ ]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target}')

when input is tensor([18]) the target: 47

when input is tensor([18, 47]) the target: 56

when input is tensor([18, 47, 56]) the target: 57

when input is tensor([18, 47, 56, 57]) the target: 58

when input is tensor([18, 47, 56, 57, 58]) the target: 1

when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15

when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47

when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58

In [ ]:
#|export
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split, batch_size = batch_size, block_size = block_size):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y        

In [ ]:
xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)
print('--------')
    
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs

torch.Size([4, 8])

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets

torch.Size([4, 8])

...

when input is [25, 17, 27, 10] the target: 0

when input is [25, 17, 27, 10, 0] the target: 21

when input is [25, 17, 27, 10, 0, 21] the target: 1

when input is [25, 17, 27, 10, 0, 21, 1] the target: 54

when input is [25, 17, 27, 10, 0, 21, 1, 54] the target: 39

In [ ]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

## Simplest Baseline: Bigram Language Model, Loss, Generation

In [ ]:
#|export
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        
        # idx and targets are both (B, T) of tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:,  -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])

tensor(5.0364, grad_fn=<NllLossBackward0>)

In [ ]:
#|export
def generate_from_the_model(m, max_new_tokens=100):
    idx = torch.zeros((1, 1), dtype=torch.long)
    print(decode(m.generate(idx, max_new_tokens=max_new_tokens)[0].tolist()))

## Training the Bigram Model

In [ ]:
@torch.no_grad()
def estimate_loss(batch_size=32, eval_iters=200):
    out = {}
    model.eval() # setting to evaluation phase
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size = batch_size)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # setting to training phase
    return out

In [ ]:
#|export
def train(model, max_iters, batch_size = 32, eval_iters = 200, eval_interval = 200):
    
    # create a PyTorch optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    
    for step in range(max_iters):

        if step % eval_interval == 0:
            losses = estimate_loss(batch_size, eval_iters)
            print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            
        # sample a batch of data
        xb, yb = get_batch('train', batch_size=batch_size)

        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    print(loss.item())

In [ ]:
#|export
model = BigramLanguageModel(vocab_size).to(device)
train(model, 15_000)

step 0: train loss 4.6317, val loss 4.6301

step 200: train loss 4.4054, val loss 4.4171

step 400: train loss 4.1955, val loss 4.2023

step 600: train loss 4.0181, val loss 4.0118

step 800: train loss 3.8346, val loss 3.8466

...

step 14200: train loss 2.4566, val loss 2.4839

step 14400: train loss 2.4573, val loss 2.4799

step 14600: train loss 2.4531, val loss 2.4867

step 14800: train loss 2.4606, val loss 2.4748

2.454706907272339

In [ ]:
generate_from_the_model(model)

ARealok t ar hyoreaceatads nss w:
ESp mofoirtre vithainof owisu canouenk;
ARDUNou; l hiluler! GENINC

In [ ]:
generate_from_the_model(model, 300)

ugof qut's, mer'de ven mbee tllathe.
Brngaro.
Bengg d swoll kive d of beav wisil aberoinor ye d, y neancour hak thithand, y t o ERRIN VOF t n apl notheasthaurrom
h, peot
ABElathod bu ar br t
Y theryessht p, hofou Wer irdouiound.

Fomailllar INus busikinthe he, athitloun hok e hamit lyomm.

There:
TI

In [ ]:
generate_from_the_model(model, 700)

HExciownthe plilonond y
Ma. le fovey dincou pakes ttherth g
TET:


TI tothed ake the tithenthiffinue t ad Bun Fo-me hifaifll pre hendak.


Thor d gaitichelleder e,
ELAREOpan INUKncrsen pe wreamo a ar weyourth,
QUMeeremy the'd, meled byounngoroll l' upeplke h ne s INRI metin,
If ce ar s y IO:
Whe hamy tod:
PUKityosof wifutha msto ingttha s, hishot yo cughel mat eamave?

ENRD:
FI be kiedHenf sme wean.
I gol, NThtl tres m?

my are hertityouor mequ ou le t thitaclliled ul br,
adine
BRUShe fe ay l; tiorou, t dr, m s mers tind,
He Clatou ie toat thy'elle t cher siryoffache y te br he pos we keat foke dy atochtarethet t, is, tthe is 
spllllirist an il atharerimat t
A: bed m mech. HMannondanthen
Deat

## Version 1
> Averaging past context with for loops, the weakest form of aggregation

### The mathematical trick in self-attention

In [ ]:
# consider the following toy example:

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [ ]:
x[0]

tensor([[-0.6693, -0.8456],
        [-0.6534,  0.8349],
        [-0.6808,  1.1112],
        [ 1.6128, -1.4807],
        [-0.5419, -0.4675],
        [ 0.1127, -0.1059],
        [-0.0641, -1.5335],
        [-0.3041,  1.6519]])

In [ ]:
xbow[0]

tensor([[-0.6693, -0.8456],
        [-0.6613, -0.0053],
        [-0.6678,  0.3669],
        [-0.0977, -0.0950],
        [-0.1865, -0.1695],
        [-0.1366, -0.1589],
        [-0.1263, -0.3553],
        [-0.1485, -0.1044]])

## Version 2
> Matrix multiply as weighted aggregation

In [ ]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('c=')
print(c)

a=

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

--

b=

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=

tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])

In [ ]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('c=')
print(c)

a=

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

--

b=

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=

tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('c=')
print(c)

a=

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

--

b=

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=

tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) --> (B, T, T) @ (B, T, C) --> (B, T, C)

In [ ]:
wei.shape

torch.Size([8, 8])

In [ ]:
xbow2.shape

torch.Size([4, 8, 2])

In [ ]:
torch.allclose(xbow2, xbow)

True

In [ ]:
xbow, xbow2

(tensor([[[-6.6931e-01, -8.4557e-01],
          [-6.6134e-01, -5.3177e-03],
          [-6.6781e-01,  3.6686e-01],
          [-9.7657e-02, -9.5026e-02],
          [-1.8651e-01, -1.6953e-01],
          [-1.3665e-01, -1.5892e-01],
          [-1.2628e-01, -3.5530e-01],
          [-1.4851e-01, -1.0440e-01]],
 
         [[ 6.1571e-01,  6.3541e-01],
          [-6.0056e-01,  3.3687e-01],
          [-3.4395e-01,  4.4428e-01],
          [-4.7228e-01,  3.2352e-01],
          [-4.1381e-01,  3.5067e-01],
          [-2.2773e-01,  1.9506e-01],
          [-3.0587e-01,  1.4499e-01],
          [ 1.2853e-01,  2.8592e-01]],
 
         [[-2.7886e-04,  4.0268e-01],
          [-1.8299e-01,  3.5538e-01],
          [-5.2246e-01,  4.5156e-01],
          [ 3.1017e-02,  3.2158e-01],
          [-1.0164e-01,  3.3109e-01],
          [-9.4260e-02,  6.2927e-02],
          [ 5.2089e-02, -7.4155e-03],
          [ 1.2877e-01, -1.4536e-01]],
 
         [[-7.1193e-01,  4.9925e-01],
          [-2.2146e-01,  1.1845e+00],
   

## Version 3:
> Adding softmax

In [ ]:
tril = torch.tril(torch.ones(T, T)); tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
wei = torch.zeros((T, T)); wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei = F.softmax(wei, dim=-1); wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

## Clean up the Model Code

In [ ]:
n_embd = 32

In [ ]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self): # removed vocab_size
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, targets = None):
        # idx and targets are both (B, T) tensor of integers
        tok_embd  = self.token_embedding_table(idx) # (B, T, C), C--> n_embd
        logits = self.lm_head(tok_embd) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:,  -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        

In [ ]:
model = BigramLanguageModel()

In [ ]:
train(model, 15_000)

step 0: train loss 4.3025, val loss 4.3075

step 200: train loss 3.1244, val loss 3.1247

step 400: train loss 2.7463, val loss 2.7792

step 600: train loss 2.6392, val loss 2.6618

step 800: train loss 2.5982, val loss 2.6138

...

step 14200: train loss 2.4632, val loss 2.4902

step 14400: train loss 2.4604, val loss 2.4828

step 14600: train loss 2.4725, val loss 2.4929

step 14800: train loss 2.4653, val loss 2.4899

2.4547626972198486

## Positional Encoding

In [ ]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logit for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, target=None):
        B, T, C = idx.shape
        
        # idx and targets are both (B, T) tenspr of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) <-- Positional Embedding
        x = tok_emb + pos_embd # (B, T, C) <--- Adding the Positional Embedding to Token Embedding
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:,  -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

## Version 4
> Self Attention

In [ ]:
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x

out.shape

torch.Size([4, 8, 32])

In [ ]:
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# lets see a single Head perform self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril((torch.ones(T, T)))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x

out.shape

torch.Size([4, 8, 32])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5553, 0.4447, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0024, 0.9931, 0.0045, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3056, 0.0135, 0.5681, 0.1128, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1010, 0.6721, 0.0109, 0.0859, 0.1300, 0.0000, 0.0000, 0.0000],
        [0.1368, 0.0013, 0.0382, 0.0324, 0.0061, 0.7852, 0.0000, 0.0000],
        [0.0440, 0.1134, 0.0289, 0.1427, 0.5070, 0.0722, 0.0917, 0.0000],
        [0.0259, 0.4333, 0.0183, 0.2266, 0.0400, 0.1100, 0.1283, 0.0176]],
       grad_fn=<SelectBackward0>)

In [ ]:
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# lets see a single Head perform self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril((torch.ones(T, T)))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x

v = value(x)
out = wei @ x

out.shape

torch.Size([4, 8, 32])

## Scaled Self Attention
> Why divide by sqrt(head_size)?

In [ ]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5

In [ ]:
k.var(), q.var(), wei.var()

(tensor(1.1065), tensor(1.1194), tensor(1.2355))

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])